## Modelo de entrenamiento para la base de datos golf

#### Importación de librerías

Importamos las librerías pandas y numpy para agilizar el manejo de la base de datos.

In [208]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statistics as stats

#### Definición de variables

##### traform_data(df)

Esta función es la encargada de transformar el `DataFrame` *`df`*. La transformación consiste en reemplazar la columna `outlook`, que contiene los valores ***sunny***, ***rainy***, y ***outcast***, por tres nuevas columnas con los mismos nombres. Los valores de estas columnas son un `1` o un `0`, dependiendo del antiguo valor de `outlook`.  

> **NOTA:** La transformación de la base de datos ha sido programada específicamente para la base de datos `golf` y cualquier otra base de datos con la misma estructura.  
> En caso de introducir una base de datos que no cumpla esta condición, el programa no arrojará los resultados esperados.  


In [209]:
def transform_data(df):
    dummies       = pd.get_dummies(df['outlook'])
    df            = pd.concat([df, dummies], axis=1)
    df            = df.drop(['outlook'], axis=1)
    cols          = df.columns.tolist()
    cols          = cols[-1 : -4 : -1] + cols[:-3]
    df[cols[:-1]] = df[cols[:-1]].astype('int64')
    return df[cols]

##### read_data(path)

Una simple función que lee una base de datos de un archivo csv desde el `path` a un `DataFrame`, y retorna la transformación de este.

In [210]:
def read_data(path):
    df = pd.read_csv(path)
    return transform_data(df)

##### split_data(df, train_size)

Función encargada de separar las filas que usaremos para el entrenamiento del modelo de las filas que usaremos para probar el modelo.  

Se recibe el `DataFrame`, y el tamaño de la muestra de entrenamiento. Seguido creamos una variable `sample_df` que almacena una muestra de `df` pero organizado de forma aleatoria por el método `df.sample(frac = 1)` .  

> **NOTA:** El método `sample()` se encarga de crear y retornar un `DataFrame` muestra, donde las filas han sido organizadas aleatoriamente. El *keyarg* `frac` representa el porcentaje de `df` que servirá como muestra. En esta ocasión está igualado a `1` ya que necesitamos todo el `DataFrame`.

Almacenamos los valores desde `0` hasta `train_size-1` de esta muestra en la variable `train`. El resto es almacenado en la variable `test`.  

Se retorna un `tuple` con ambos `DataFrames`.

In [211]:
def split_data(df, train_size):
    sample_df = df.sample(frac=1)
    train     = sample_df[ : train_size]
    test      = sample_df[train_size : ]
    return train, test

##### euclidean_distance(p, q)

Función encargada de calcular la distancia euclidiana entre la fila test `p` y la fila train `q`.  

Se reciben dos `numpy arrays` y se excluye el último valor de ambos, el cual representa el valor de `play`.
Dado que son `numpy array` los arreglos pueden ser tratados como vectores euclidianos, por lo que hacemos la sustracción, y la elevación al cuadrado de forma directa. 

Esta operación retorna un nuevo `numpy array`, el cual es pasado como parámetro a la función `np.sum()`. `np.sum()` retorna la suma de todos los valores que se encuentran dentro de el *array*. El resultado de esta suma es pasado a la función `np.sqrt()` para calcular su raíz cuadrada.  

Finalmente, el valor de la raíz cuadrada es retornado.

In [212]:
def euclidean_distance(p, q):
    x = np.array(p.values)[:-1]
    y = np.array(q.values)[:-1]
    return np.sqrt(np.sum((y - x) ** 2))

##### kNearestNeighbors(x, train_data, k)

Función encargada de retornar los vecinos más cercanos a `x`.  

Se crea un arreglo `neighbors` usando un técnica llamada *list comprehension*. El tamaño del arreglo es igual al tamaño de `train_data`.   
Esta variable almacena un arreglo de *tuples*, los cuales tienen como primer elemento la distancia entre `p` y `q_i`. El segundo elemento de cada `tuple` es el índice de `q`.  

El arreglo se organiza de forma ascendente basándose en los valores de las distancias euclidianas y finalmente se retornan los primeros `k` elementos del arreglo, los cuales representan los `k` vecinos más cercanos.

In [213]:
def kNearestNeighbors(p, train_data, k):
    neighbors = [(euclidean_distance(p, q), i) for i, q in train_data.iterrows()]    
    neighbors.sort()
    return neighbors[ :k]

In [214]:
def train(k, train_data, test_data):
    errors = 0.0
    for i, x in test_data.iterrows():
        y          = kNearestNeighbors(x, train_data, k)
        results    = df.iloc[[tup[1] for tup in y]]['play'].tolist()
        prediction = stats.mode(results)

        print(f'Punto: {x.values, i} \nVecinos: {y} \nPredicción: {prediction}\n\n')

        if prediction != x[-1]:
            errors += 1
        
    return errors / len(test_data)


In [215]:
df = read_data('golf.csv')
train_data, test_data = split_data(df, 10)
print(train_data)
print(test_data)

    sunny  rainy  overcast  temperature  humidity  windy play
9       0      1         0           75        80      0  yes
2       0      0         1           83        86      0  yes
6       0      0         1           64        65      1  yes
13      0      1         0           71        91      1   no
12      0      0         1           81        75      0  yes
0       1      0         0           85        85      0   no
5       0      1         0           65        70      1   no
11      0      0         1           72        90      1  yes
1       1      0         0           80        90      1   no
4       0      1         0           68        80      0  yes
    sunny  rainy  overcast  temperature  humidity  windy play
10      1      0         0           75        70      1  yes
8       1      0         0           69        70      0  yes
3       0      1         0           70        96      0  yes
7       1      0         0           72        95      0   no


In [216]:
train(3, train_data, test_data)

Punto: (array([1, 0, 0, 75, 70, 1, 'yes'], dtype=object), 10) 
Vecinos: [(8.0, 12), (10.099504938362077, 5), (10.14889156509222, 9)] 
Predicción: yes


Punto: (array([1, 0, 0, 69, 70, 0, 'yes'], dtype=object), 8) 
Vecinos: [(4.358898943540674, 5), (7.280109889280518, 6), (10.14889156509222, 4)] 
Predicción: yes


Punto: (array([0, 1, 0, 70, 96, 0, 'yes'], dtype=object), 3) 
Vecinos: [(5.196152422706632, 13), (6.557438524302, 11), (11.789826122551595, 1)] 
Predicción: no


Punto: (array([1, 0, 0, 72, 95, 0, 'no'], dtype=object), 7) 
Vecinos: [(4.47213595499958, 13), (5.291502622129181, 11), (9.486832980505138, 1)] 
Predicción: no




0.25

In [217]:
train(5, train_data, test_data)

Punto: (array([1, 0, 0, 75, 70, 1, 'yes'], dtype=object), 10) 
Vecinos: [(8.0, 12), (10.099504938362077, 5), (10.14889156509222, 9), (12.165525060596439, 6), (12.328828005937952, 4)] 
Predicción: yes


Punto: (array([1, 0, 0, 69, 70, 0, 'yes'], dtype=object), 8) 
Vecinos: [(4.358898943540674, 5), (7.280109889280518, 6), (10.14889156509222, 4), (11.74734012447073, 9), (13.076696830622021, 12)] 
Predicción: yes


Punto: (array([0, 1, 0, 70, 96, 0, 'yes'], dtype=object), 3) 
Vecinos: [(5.196152422706632, 13), (6.557438524302, 11), (11.789826122551595, 1), (16.1245154965971, 4), (16.46207763315433, 2)] 
Predicción: yes


Punto: (array([1, 0, 0, 72, 95, 0, 'no'], dtype=object), 7) 
Vecinos: [(4.47213595499958, 13), (5.291502622129181, 11), (9.486832980505138, 1), (14.2828568570857, 2), (15.362291495737216, 9)] 
Predicción: yes




0.25

In [218]:
train(7, train_data, test_data)

Punto: (array([1, 0, 0, 75, 70, 1, 'yes'], dtype=object), 10) 
Vecinos: [(8.0, 12), (10.099504938362077, 5), (10.14889156509222, 9), (12.165525060596439, 6), (12.328828005937952, 4), (17.97220075561143, 2), (18.05547008526779, 0)] 
Predicción: yes


Punto: (array([1, 0, 0, 69, 70, 0, 'yes'], dtype=object), 8) 
Vecinos: [(4.358898943540674, 5), (7.280109889280518, 6), (10.14889156509222, 4), (11.74734012447073, 9), (13.076696830622021, 12), (20.29778313018444, 11), (21.166010488516726, 13)] 
Predicción: yes


Punto: (array([0, 1, 0, 70, 96, 0, 'yes'], dtype=object), 3) 
Vecinos: [(5.196152422706632, 13), (6.557438524302, 11), (11.789826122551595, 1), (16.1245154965971, 4), (16.46207763315433, 2), (16.76305461424021, 9), (18.65475810617763, 0)] 
Predicción: yes


Punto: (array([1, 0, 0, 72, 95, 0, 'no'], dtype=object), 7) 
Vecinos: [(4.47213595499958, 13), (5.291502622129181, 11), (9.486832980505138, 1), (14.2828568570857, 2), (15.362291495737216, 9), (15.588457268119896, 4), (16.4012194

0.25